In [1]:
import numpy as np
import sympy as sp

In [17]:
class Node:
    def __init__(self, operation=None, left=None, right=None, value=None):
        self.operation = operation
        self.left = left
        self.right = right
        self.value = value
        
    def evaluate(self):
        if self.operation is None:
            return self.value
        elif self.operation == '+':
            return self.left.evaluate() + self.right.evaluate()
        elif self.operation == '*':
            return self.left.evaluate() * self.right.evaluate()
        
        
    def __repr__(self):
        """노드 정보 보기 쉽게 출력"""
        if self.operation is None:
            return f"({self.value})"
        return f"({self.left} {self.operation} {self.right})"
        

In [21]:
input_nodes = [
    Node(value=3),
    Node(value=2),
    Node(value=1),
    Node(value=7),
    Node(value=5),
    Node(value=4),
]

In [22]:
gate1 = Node(operation='*', left=input_nodes[0], right=input_nodes[1])
gate2 = Node(operation='*', left=gate1, right=Node(operation='+', left=input_nodes[2], right=input_nodes[3]))
gate3 = Node(operation='*', left=Node(operation='+', left=input_nodes[2], right=input_nodes[3]),  right=Node(operation='+', left=input_nodes[4], right=input_nodes[5]))

In [23]:
print("Gate Structure:")
print(f"Gate 1: {gate1}")
print(f"Gate 2: {gate2}")
print(f"Gate 3: {gate3}")

print("\nEvaluations:")
print(f"Gate 1 Result: {gate1.evaluate()}")
print(f"Gate 2 Result: {gate2.evaluate()}")
print(f"Gate 3 Result: {gate3.evaluate()}")

Gate Structure:
Gate 1: ((3) * (2))
Gate 2: (((3) * (2)) * ((1) + (7)))
Gate 3: (((1) + (7)) * ((5) + (4)))

Evaluations:
Gate 1 Result: 6
Gate 2 Result: 48
Gate 3 Result: 72


In [82]:
transcript_table = [3, 2, 1, 7, 5, 4, 6, 48, 72]
gate_list = [gate1, gate2, gate3]


def find_left_selector(gate, gate_idx, c, selector):    
    if gate.operation == None:
        if gate.evaluate() == c:
            selector[gate_idx] = 1        
            print(f"gate_idx: {gate_idx}, c: {c}, selector: {selector}")    
    elif gate.left.operation == None:
        if gate.left.evaluate() == c:
            selector[gate_idx] = 1        
            print(f"gate_idx: {gate_idx}, c: {c}, selector: {selector}")
    elif gate.left.operation == '+':
        find_left_selector(gate.left.left, gate_idx, c, selector)
        find_left_selector(gate.left.right, gate_idx, c, selector)
    elif gate.operation == '*':
        if gate.left.evaluate() == c:
            selector[gate_idx] = 1        
            print(f"gate_idx: {gate_idx}, c: {c}, selector: {selector}")

    return selector

def find_right_selector(gate, gate_idx, c, selector):    
    if gate.operation == None:
        if gate.evaluate() == c:
            selector[gate_idx] = 1        
            print(f"gate_idx: {gate_idx}, c: {c}, selector: {selector}")    
    elif gate.right.operation == None:
        if gate.right.evaluate() == c:
            selector[gate_idx] = 1        
            print(f"gate_idx: {gate_idx}, c: {c}, selector: {selector}")
    elif gate.right.operation == '+':
        find_right_selector(gate.right.right, gate_idx, c, selector)
        find_right_selector(gate.right.left, gate_idx, c, selector)
    elif gate.operation == '*':
        if gate.right.evaluate() == c:
            selector[gate_idx] = 1        
            print(f"gate_idx: {gate_idx}, c: {c}, selector: {selector}")

    return selector

def gate_selector(gate_list, direction, transcript_table):
    polynomial_table = []
    if direction == 'left':
        for c in transcript_table:
            selector = [0] * len(gate_list)
            for gate_idx, gate in enumerate(gate_list):
                find_left_selector(gate, gate_idx, c, selector)
            polynomial_table.append(selector)
            
    elif direction == 'right':
        for c in transcript_table:
            selector = [0] * len(gate_list)
            for gate_idx, gate in enumerate(gate_list):
                find_right_selector(gate, gate_idx, c, selector)
            polynomial_table.append(selector)
            
    return polynomial_table


In [83]:
gate_selector(gate_list, 'left', transcript_table)

gate_idx: 0, c: 3, selector: [1, 0, 0]
gate_idx: 2, c: 1, selector: [0, 0, 1]
gate_idx: 2, c: 7, selector: [0, 0, 1]
gate_idx: 1, c: 6, selector: [0, 1, 0]


[[1, 0, 0],
 [0, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 0],
 [0, 1, 0],
 [0, 0, 0],
 [0, 0, 0]]

In [84]:
gate_selector(gate_list, 'right', transcript_table)

gate_idx: 0, c: 2, selector: [1, 0, 0]
gate_idx: 1, c: 1, selector: [0, 1, 0]
gate_idx: 1, c: 7, selector: [0, 1, 0]
gate_idx: 2, c: 5, selector: [0, 0, 1]
gate_idx: 2, c: 4, selector: [0, 0, 1]


[[0, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0]]

In [115]:
def find_selector(gate, gate_idx, c, selector, direction):
    """Left 또는 Right Selector Polynomial을 찾는 함수"""
    if gate is None:
        return
    
    # 리프 노드라면 값을 확인
    if gate.operation is None:
        if gate.evaluate() == c:
            selector[gate_idx] = 1        
        return
    
    # 왼쪽 / 오른쪽 / 출력 선택
    if direction == 'left':
        target = gate.left 
    elif direction == 'right':
        target = gate.right
    elif direction == 'output':
        target = gate

    # 리프 값이 맞는 경우
    if target and target.operation is None and target.evaluate() == c:
        selector[gate_idx] = 1
        return

    # 덧셈 노드인 경우 재귀 호출
    if target and target.operation == '+':
        find_selector(target.left, gate_idx, c, selector, direction)
        find_selector(target.right, gate_idx, c, selector, direction)

    # 곱셈 노드이면서 직접 값이 맞는 경우
    if gate.operation == '*' and target.evaluate() == c:
        selector[gate_idx] = 1

def gate_selector(gate_list, direction, transcript_table):
    """게이트에서 left 또는 right selector polynomial을 찾는 함수"""
    polynomial_table = []
    for c in transcript_table:
        selector = [0] * len(gate_list)
        for gate_idx, gate in enumerate(gate_list):
            find_selector(gate, gate_idx, c, selector, direction)
        polynomial_table.append(selector)
    return polynomial_table



In [116]:

# 실행 예시 (left selector 생성)
left_selector_table = gate_selector(gate_list, 'left', transcript_table)
print("Left Selector Table:")
left_selector_table

Left Selector Table:


[[1, 0, 0],
 [0, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 0],
 [0, 1, 0],
 [0, 0, 0],
 [0, 0, 0]]

In [117]:

# 실행 예시 (left selector 생성)
right_selector_table = gate_selector(gate_list, 'right', transcript_table)
print("Right Selector Table:")
right_selector_table

Right Selector Table:


[[0, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0]]

In [118]:

# 실행 예시 (left selector 생성)
output_selector_table = gate_selector(gate_list, 'output', transcript_table)
print("output Selector Table:")
output_selector_table

output Selector Table:


[[0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 0, 1]]